# Task02. 搭建开发环境并运行、理解时序插补工作流

## 1. 开发环境配置

### PyPOTS开发环境支持多种安装方式, 你可以自由选择从源码安装, 通过pip安装PyPI上的发布版本或者使用conda从conda-forge的发行版进行环境配置, 如果你熟悉docker的使用方式, 也可以通过docker来获取我们已经为你配置好的PyPOTS开发环境容器

### 从下方选择一种你熟悉的安装方式来为PyPOTS配置Python开发环境 (这里我们指定了各个安装方式都安装v0.18版本, 因为该版本是本次课程编排时的最新版本, 为防止以后新版本释出导致与教程的不兼容, 所以我们在此固定版本为v0.18)

💡 **补充说明**：不同的安装方式适用于不同的使用场景。

- 从源码安装适合希望获取最新特性或参与开发的用户。
- pip 安装适合大多数用户，获取的是官方发布的稳定版本。
- conda 安装适合在 Anaconda 环境中使用，依赖管理更方便。
- docker 安装适合希望一键部署环境的用户，尤其适合在服务器上部署运行。

In [35]:
# 从源码安装
# 🚀 从源码安装 PyPOTS：推荐用于获取最新开发版（可能包含最新特性，但稳定性略低）
# 如果你希望参与开发或想尝试最新功能，可以使用这种方式安装
!pip install https://github.com/WenjieDu/PyPOTS/archive/refs/tags/v0.18.zip

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     / 987.4 kB 435.3 kB/s 0:00:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [34]:
# 从PyPI安装 
# ✅ 从PyPI安装 PyPOTS：推荐用于稳定版本的安装
# 这种方式适合大多数用户，安装的是在 PyPI 上发布的最新稳定版本
!pip install pypots==0.18

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [36]:
# 从conda-forge安装 (‼️请确定你熟悉conda的操作并且确认你的电脑上安装了conda)
!conda install conda-forge::pypots=0.18

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/anaconda3/envs/ml

  added / updated specs:
    - conda-forge::pypots=0.18


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-2.1.0              |  py312hca03da5_0         248 KB  defaults
    aiohappyeyeballs-2.4.4     |  py312hca03da5_0          28 KB  defaults
    aiohttp-3.11.10            |  py312h80987f9_0         910 KB  defaults
    arrow-cpp-19.0.0           |       h0b7d223_2         8.3 MB  defaults
    attrs-24.3.0               |  py312hca03da5_0         171 KB  defaults
    benchpots-0.4              |     pyhd8ed1ab_1          23 KB  conda-forge
    bottleneck-1.4.2           |  py312ha86b861_0         124 KB  defaults
    brotli-python-1.0.9        |  py312h313beb8_9         367 KB  defa

In [ ]:
# 运行配置好PyPOTS开发环境的docker容器 (‼️请确定你熟悉docker的使用并且确认你的电脑上安装了docker)
!docker run -it --name pypots wenjiedu/pypots

## 2. 时间序列插补工作流

## 生成一个随机的时间序列数据集

In [37]:
# 导入必要的库
from benchpots.datasets import preprocess_physionet2012  # 导入physionet2012数据集预处理函数

# 加载并预处理physionet2012数据集
# subset="set-a": 使用set-a子集
# pattern="point": 使用点缺失模式
# rate=0.1: 设置缺失率为10%
physionet2012_dataset = preprocess_physionet2012(
    subset="set-a", 
    pattern="point", 
    rate=0.1,
)

# 打印数据集的键，查看包含哪些数据
print(physionet2012_dataset.keys())

2025-05-10 23:34:22 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-10 23:34:22 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-10 23:34:22 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-10 23:34:22 [INFO]: Loaded successfully!
2025-05-10 23:34:28 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-10 23:34:28 [INFO]: 23249 values masked out in the val set as ground truth, take 10.03% of the original observed values
2025-05-10 23:34:28 [INFO]: 29282 values masked out in the test set as ground truth, take 10.05% of the original observed values
2025-05-10 23:34:28 [INFO]: Total sample number: 3997
2025-05-10 23:34:28 [I

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


In [38]:
# 导入numpy库用于数值计算
import numpy as np

# 创建测试集的缺失值指示掩码
# 通过异或运算(^)比较原始数据和缺失数据的NaN位置，得到真实缺失值的位置
physionet2012_dataset["test_X_indicating_mask"] = np.isnan(physionet2012_dataset["test_X"]) ^ np.isnan(physionet2012_dataset["test_X_ori"])

# 将原始测试数据中的NaN值替换为0
physionet2012_dataset["test_X_ori"] = np.nan_to_num(physionet2012_dataset["test_X_ori"])

# 构建训练集字典，只包含输入特征X
train_set = {
    "X": physionet2012_dataset["train_X"],
}

# 构建验证集字典，包含输入特征X和原始数据X_ori
val_set = {
    "X": physionet2012_dataset["val_X"],
    "X_ori": physionet2012_dataset["val_X_ori"],
}

# 构建测试集字典，包含输入特征X和原始数据X_ori
test_set = {
    "X": physionet2012_dataset["test_X"],
    "X_ori": physionet2012_dataset["test_X_ori"],
}

In [39]:
# 导入SAITS模型，这是一个用于时间序列数据插补的深度学习模型
from pypots.imputation import SAITS

# 初始化SAITS模型，设置模型参数
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],      # 时间步长，从数据集中获取
    n_features=physionet2012_dataset['n_features'], # 特征数量，从数据集中获取
    n_layers=3,                                     # Transformer编码器层数
    d_model=64,                                     # 模型维度
    n_heads=4,                                      # 注意力头数
    d_k=16,                                         # 每个注意力头的键维度
    d_v=16,                                         # 每个注意力头的值维度
    d_ffn=128,                                      # 前馈神经网络隐藏层维度
    dropout=0.1,                                    # Dropout比率，用于防止过拟合
    # 你可以调整参数ORT_weight和MIT_weight的权重值，以使SAITS模型更多地关注于一个任务。通常你可以让它们保持默认值，比如1
    ORT_weight=1,
    MIT_weight=1,
    batch_size=32,
    # 这里为了快速演示我们将epochs设置为10，你可以将其设置为100或更多以获得更好的结果
    epochs=10,
    # 这里我们设置patience=3，如果连续3个epoch的评估loss没有减少，则提前停止训练。你可以不设置它,则默认为None,禁用早停机制
    patience=3,
    # 设置优化器。不同于torch.optim。在初始化pypots.optimizer时，你不必指定模型的参数。您也可以不设置它, 它将默认初始化一个lr=0.001的Adam优化器。
    optimizer=Adam(lr=1e-3),
    # 这个num_workers参数用于torch.utils.data.Dataloader。它是用于数据加载的子进程的数量。让它默认为0意味着数据加载将在主进程中，即不会有子进程。如果你认为数据加载是模型训练速度的瓶颈，则可以将其增加
    num_workers=0,
    # 如果不设置device, PyPOTS将自动为你分配最佳设备。这里我们将其设置为“cpu”。你也可以设置为'cuda', ‘cuda:0’或‘cuda:1’，如果你有多个cuda设备，甚至并行['cuda:0', 'cuda:1']
    device=DEVICE,
    # 设置保存tensorboard和训练模型文件的路径
    saving_path="result_saving/imputation/saits",
    # 训练完成后只保存最好的模型。你还可以将其设置为“better”，以保存在训练期间每一次在val set上表现得比之前更好的模型
    model_saving_strategy="best",
)

2025-05-10 23:34:28 [INFO]: No given device, using default device: cpu
2025-05-10 23:34:28 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 23:34:28 [INFO]: Using customized MAE as the training loss function.
2025-05-10 23:34:28 [INFO]: Using customized MSE as the validation metric function.
2025-05-10 23:34:28 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 218,294


In [40]:
# 使用训练集和验证集对SAITS模型进行训练
# train_set: 包含训练数据的字典，其中"X"键对应的值包含输入特征
# val_set: 包含验证数据的字典，其中"X"键对应的值包含输入特征，"X_ori"键对应的值包含原始数据
# 训练过程中会自动使用之前设置的参数，包括epochs=10等超参数
saits.fit(train_set, val_set)

2025-05-10 23:34:38 [INFO]: Epoch 001 - training loss (MAE): 1.0883, validation MSE: 0.5367
2025-05-10 23:34:48 [INFO]: Epoch 002 - training loss (MAE): 0.7857, validation MSE: 0.4356
2025-05-10 23:34:58 [INFO]: Epoch 003 - training loss (MAE): 0.6906, validation MSE: 0.4155
2025-05-10 23:35:06 [INFO]: Epoch 004 - training loss (MAE): 0.6471, validation MSE: 0.3987
2025-05-10 23:35:13 [INFO]: Epoch 005 - training loss (MAE): 0.6151, validation MSE: 0.3893
2025-05-10 23:35:21 [INFO]: Epoch 006 - training loss (MAE): 0.5957, validation MSE: 0.3855
2025-05-10 23:35:30 [INFO]: Epoch 007 - training loss (MAE): 0.5764, validation MSE: 0.3746
2025-05-10 23:35:40 [INFO]: Epoch 008 - training loss (MAE): 0.5647, validation MSE: 0.3758
2025-05-10 23:35:49 [INFO]: Epoch 009 - training loss (MAE): 0.5506, validation MSE: 0.3664
2025-05-10 23:35:58 [INFO]: Epoch 010 - training loss (MAE): 0.5416, validation MSE: 0.3590
2025-05-10 23:35:58 [INFO]: Finished training. The best model is from epoch#10.


In [41]:
# 使用训练好的SAITS模型对测试集进行预测，生成缺失值填充结果
# test_set: 包含测试数据的字典，其中"X"键对应的值包含输入特征
# 返回的test_set_imputation_results是一个字典，其中重要的键值对为：
# - "imputation": 填充后的完整时间序列数据，形状为(n_samples, n_steps, n_features)
test_set_imputation_results = saits.predict(test_set)

In [42]:
# 从pypots库中导入计算均方误差(MSE)的函数
from pypots.nn.functional import calc_mse

# 计算测试集上的均方误差(MSE)
# test_set_imputation_results["imputation"]: SAITS模型对测试集的插补结果
# physionet2012_dataset["test_X_ori"]: 测试集的原始完整数据
# physionet2012_dataset["test_X_indicating_mask"]: 测试集的缺失值指示掩码
test_MSE = calc_mse(
    test_set_imputation_results["imputation"],
    physionet2012_dataset["test_X_ori"],
    physionet2012_dataset["test_X_indicating_mask"],
)
# 打印SAITS模型在测试集上的MSE评估结果
print(f"SAITS test_MSE: {test_MSE}")

SAITS test_MSE: 0.32843214033958174


In [43]:
from pypots.data.saving import pickle_dump

train_set_imputation = saits.impute(train_set)
val_set_imputation = saits.impute(val_set)
test_set_imputation = test_set_imputation_results["imputation"]

dict_to_save={
    'train_set_imputation': train_set_imputation,
    'train_set_labels': physionet2012_dataset['train_y'],
    'val_set_imputation': val_set_imputation,
    'val_set_labels': physionet2012_dataset['val_y'],
    'test_set_imputation': test_set_imputation,
    'test_set_labels': physionet2012_dataset['test_y'],
}

pickle_dump(dict_to_save, "result_saving/imputed_physionet2012.pkl")

2025-05-10 23:36:05 [INFO]: Successfully saved to result_saving/imputed_physionet2012.pkl


# 3. 阅读材料

### Du, W., Cote, D., & Liu, Y. (2023). [SAITS: Self-Attention-based Imputation for Time Series](https://arxiv.org/abs/2202.08516). *Expert systems with applications*.
#### 推荐原因: 该文率先将自注意机制引入时序插补领域, 提出了基于门控的加权融合架构以及基于观测重构任务(ORT)和掩码插补任务(MIT)联合训练的方法. 文章于2023年被人工智能顶级期刊ESWA收录. 截止2025年5月Google Scholar上引用300+.